<a href="https://colab.research.google.com/github/ffsf-filho/horta_sustentavel_residencial/blob/main/horta_sustentavel_residencial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instala os SDKs necessários:
!pip install -q -U google-generativeai
!pip install gradio

In [2]:
# Importa as bibliotecas necessárias:
import google.generativeai as genai  # Biblioteca do Google para modelos de linguagem
import gradio as gr  # Biblioteca para criar interfaces web interativas
from google.colab import userdata # Biblioteca para acessar dados do usuário no Colab

# Configura a chave da API do Google:
GOOGLE_API_KEY= userdata.get('SECRET_API_GEMINI')
genai.configure(api_key=GOOGLE_API_KEY)

# Listando os modelos disponíveis
# for m in genai.list_models():
#   if 'generateContent' in m.supported_generation_methods:
#     print(m.name)

In [3]:
# Define o modelo de linguagem a ser usado (Gemini Pro):
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
}
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

In [4]:
# Usa o modelo de linguagem para gerar texto explicando como cultivar a planta em um vaso:
response = model.generate_content(f"Explain in detail how to cultivate Alface in a pot.")
# Retorna o texto gerado:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': "**Materials:**\n\n* Pot with drainage holes (at least 12 inches in diameter)\n* Potting mix specifically designed for vegetables\n* Lettuce seeds or seedlings\n* Watering can or hose\n* Liquid fertilizer (optional)\n\n**Steps:**\n\n**1. Choose a Suitable Pot:**\n\n* Select a pot that is at least 12 inches in diameter and has drainage holes.\n* The pot should be deep enough to accommodate the lettuce roots, which can grow up to 6 inches long.\n* Terracotta or plastic pots are both suitable options.\n\n**2. Prepare the Potting Mix:**\n\n* Fill the pot with a high-quality potting mix specifically designed for vegetables.\n* The potting mix should be well-draining and rich in organic matter.\n* Amend the potting mix with compost or manure to improve soil fertility.\n\n**3. Plant the Lettuce:**\n\n* **Seeds:** Sow lettuce seeds 1/4 inch

In [7]:
# Define uma função para obter informações sobre o cultivo de uma planta:
def getInfoCultivo(plant_type):
    # Usa o modelo de linguagem para gerar texto explicando como cultivar a planta em um vaso:
    response = model.generate_content(f"explique detalhadamente como cultivar {plant_type} em um vaso, indicando qual o tamanho ideal do vaso e o tempo de colheita.")
    # Retorna o texto gerado:
    return response.text

# Define estilos CSS para a interface web:
style = """
  .gradio-container {
      background-color: #ccff99;
      color: blue;
  }
  .folhas-section {
      background-color: #bfff80;
  }
  .raizes-section {
      background-color: #ecd9c6;
  }
  .frutos-section {
      background-color: #ffffb3;
  }
  .vagens-section {
      background-color: #b3ffb3;
  }
  .ervas-section {
      background-color: #e0e0d1;
  }
  .frutas-section {
      background-color: #ffad99;
  }
"""

# Define listas de plantas para cada categoria:
folhas=['Alface', 'Rúcula', 'Espinafre', 'Couve', 'Acelga', 'Agrião']
raizes=['Cenoura', 'Rabanete', 'Beterraba']
frutos=['Tomate cereja', 'Pimentão', 'Pimenta', 'Berinjela']
vagens=['Ervilha', 'Feijão-vagem']
ervas=['Manjericão', 'salsinha', 'cebolinha', 'coentro', 'hortelã', 'orégano', 'alecrim', 'tomilho']
frutas=['Jaboticaba','Laraja anã', 'Limão anã', 'Morango', 'Tanegrina anã']

# Cria a interface web usando Gradio:
with gr.Blocks(css=style) as interface:# Add background image
    # Adiciona um título:
    gr.Markdown("# Horta em Casa: Um Guia para Cultivo em Vasos")

    # Adiciona uma breve descrição:
    gr.Markdown("## Descubra como cultivar seus próprios alimentos frescos e saudáveis em casa, mesmo com espaço limitado!")

    # Cria uma linha com duas colunas:
    with gr.Row():
        # Cria a coluna da esquerda para as categorias de plantas:
        with gr.Column(scale=1, min_width=300):
             # Cria um Accordion para as categorias de plantas:
            with gr.Accordion("Horta em Vasos", open=True):
                # Cria abas para cada categoria de planta:
                with gr.TabItem("Folhas",elem_classes='folhas-section'):
                    # Cria um botão de rádio para escolher uma folha:
                    radio_folhas = gr.Radio(folhas, label="Escolha uma folha:",elem_classes='folhas-section')
                with gr.TabItem("Raízes", elem_classes='raizes-section'):
                    radio_raizes = gr.Radio(raizes, label="Escolha uma raiz:", elem_classes='raizes-section')
                with gr.TabItem("Frutos", elem_classes='frutos-section'):
                    radio_frutos = gr.Radio(frutos, label="Escolha um fruto:", elem_classes='frutos-section')
                with gr.TabItem("Vagens", elem_classes='frutos-section'):
                    radio_vagens = gr.Radio(vagens, label="Escolha uma Vage:", elem_classes='vagens-section')
                with gr.TabItem("Ervas", elem_classes='Ervas-section'):
                    radio_ervas = gr.Radio(ervas, label="Escolha uma erva:", elem_classes='ervas-section')
                with gr.TabItem("Frutas", elem_classes='frutas-section'):
                    radio_frutas = gr.Radio(frutas, label="Escolha uma erva:", elem_classes='frutas-section')

    # Cria uma caixa de texto para exibir as informações de cultivo:
    cultivation_info = gr.Textbox(label="Forma de Cultivo", lines=5)

    # Conecta os botões de rádio à função getInfoCultivo:
    radio_folhas.change(getInfoCultivo, inputs=radio_folhas, outputs=cultivation_info)
    radio_raizes.change(getInfoCultivo, inputs=radio_raizes, outputs=cultivation_info)
    radio_frutos.change(getInfoCultivo, inputs=radio_frutos, outputs=cultivation_info)
    radio_vagens.change(getInfoCultivo, inputs=radio_vagens, outputs=cultivation_info)
    radio_ervas.change(getInfoCultivo, inputs=radio_ervas, outputs=cultivation_info)
    radio_frutas.change(getInfoCultivo, inputs=radio_frutas, outputs=cultivation_info)

# Inicia a interface web:
interface.launch(debug=True)
#interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6bd426176da29e5074.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6bd426176da29e5074.gradio.live
